# 构建知识图谱

In [1]:
import os
import json
from py2neo import Graph, Node
import pandas as pd

In [48]:
# 注意，这里的用户名为neo4j全局用户名，而非DBMS或者database的名称
#连接数据库
g = Graph('bolt://localhost:你的端口', auth=('neo4j', '你的密码'))

In [4]:
df = pd.read_csv('medical_data.csv')

In [5]:
#症状实体提取
symptoms = []
for each in df['症状']:
    symptoms.extend(each.split(','))
symptoms = set(symptoms)

In [6]:
#科室实体提取
departments = []
for each in df['科室']:
    departments.extend(each.split(','))
departments = set(departments)

In [7]:
#检查提取
checks = []
for each in df['检查']:
    checks.extend(each.split(','))
checks = set(checks)

In [8]:
#药物提取
drugs = []
for each in df['推荐药物']:
    try:
        drugs.extend(each.split(','))
    except:
        pass
for each in df['常用药物']:
    try:
        drugs.extend(each.split(','))
    except:
        pass
drugs = set(drugs)

In [9]:
#食物提取
foods = []
for each in df['可以吃']:
    try:
        foods.extend(each.split(','))
    except:
        pass
for each in df['不可以吃']:
    try:
        foods.extend(each.split(','))
    except:
        pass
for each in df['推荐吃']:
    try:
        foods.extend(each.split(','))
    except:
        pass
foods = set(foods)

In [10]:
#药厂提取
producers = []

for each in df['具体药物']:
    try:
        for each_drug in each.split(','):
            producer = each_drug.split('(')[0]
            producers.append(producer)
    except:
        pass
producers = set(producers)

In [11]:
#疾病信息提取为字典
disease_infos = [] # 疾病信息
for idx, row in df.iterrows():
    disease_infos.append(dict(row))

In [12]:
#属性名
dict(row).keys()

dict_keys(['疾病名称', '疾病描述', '疾病种类', '科室', '病因', '症状', '检查', '并发症', '花费', '疗程', '疗法', '治愈率', '易感人群', '感染概率', '感染途径', '预防措施', '推荐药物', '常用药物', '具体药物', '可以吃', '不可以吃', '推荐吃', '是否纳入医保'])

In [13]:
#关系
def deduplicate(rels_old):
    '''关系去重函数'''
    rels_new = []
    for each in rels_old:
        if each not in rels_new:
            rels_new.append(each)
    return rels_new

In [14]:
#关系：疾病-检查
rels_check = []
for idx, row in df.iterrows():
    for each in row['检查'].split(','):
        rels_check.append([row['疾病名称'], each])
rels_check = deduplicate(rels_check)

In [15]:
rels_check

[['肺泡蛋白质沉积症', '胸部CT检查'],
 ['肺泡蛋白质沉积症', '肺活检'],
 ['肺泡蛋白质沉积症', '支气管镜检查'],
 ['百日咳', '耳、鼻、咽拭子细菌培养'],
 ['百日咳', '周围血白细胞计数及分类检验'],
 ['百日咳', '血常规'],
 ['百日咳', '酶联免疫吸附试验'],
 ['百日咳', '白细胞分类计数'],
 ['苯中毒', '血常规'],
 ['苯中毒', '骨髓象分析'],
 ['苯中毒', '先令氏指数'],
 ['喘息样支气管炎', '肺部检查'],
 ['喘息样支气管炎', '肺和胸膜听诊'],
 ['喘息样支气管炎', '抗链球菌型M蛋白抗体'],
 ['喘息样支气管炎', '抗链球菌壁多糖抗体'],
 ['喘息样支气管炎', '酶联免疫吸附试验'],
 ['成人呼吸窘迫综合征', '胸部CT检查'],
 ['成人呼吸窘迫综合征', '呼吸肌功能测定'],
 ['成人呼吸窘迫综合征', '血浆蛋白C抗原'],
 ['成人呼吸窘迫综合征', '肺泡气-动脉血氧分压差'],
 ['成人呼吸窘迫综合征', '肺毛细血管楔压'],
 ['大量羊水吸入', '肺部检查'],
 ['大量羊水吸入', '胸部透视'],
 ['大量羊水吸入', '胸部平片'],
 ['大量羊水吸入', '胸部CT检查'],
 ['单纯性肺嗜酸粒细胞浸润症', '痰液中细胞分类'],
 ['单纯性肺嗜酸粒细胞浸润症', '胸部平片'],
 ['单纯性肺嗜酸粒细胞浸润症', '痰液中寄生虫和虫卵'],
 ['大叶性肺炎', 'Optochin敏感试验'],
 ['大叶性肺炎', '小白鼠毒力试验'],
 ['大叶性肺炎', '痰培养'],
 ['大叶性肺炎', '肺活量体重指数'],
 ['大叶性肺炎', '胸部平片'],
 ['大叶性肺炎', '免疫电泳'],
 ['大叶性肺炎', '血常规'],
 ['大叶性肺炎', '痰液细菌涂片检查'],
 ['大楼病综合征', '钼靶X线检查'],
 ['大楼病综合征', 'CT检查'],
 ['大楼病综合征', '血常规'],
 ['二硫化碳中毒', '血常规'],
 ['二硫化碳中毒', '尿常规'],
 ['二硫化碳中毒', '肾功能检查'],
 ['二硫化碳中毒', '神经系统检查

In [16]:
#关系：疾病-症状
rels_symptom = []
for idx, row in df.iterrows():
    for each in row['症状'].split(','):
        rels_symptom.append([row['疾病名称'], each])
rels_symptom = deduplicate(rels_symptom)

In [17]:
#关系：疾病-疾病（并发症）
rels_acompany = []
for idx, row in df.iterrows():
    for each in row['并发症'].split(','):
        rels_acompany.append([row['疾病名称'], each])
rels_acompany = deduplicate(rels_acompany)

In [18]:
#关系：疾病-推荐药物
rels_recommanddrug = []
for idx, row in df.iterrows():
    try:
        for each in row['推荐药物'].split(','):
            rels_recommanddrug.append([row['疾病名称'], each])
    except:
        pass
rels_recommanddrug = deduplicate(rels_recommanddrug)

In [19]:
#关系：疾病-常用药物
rels_commonddrug = []
for idx, row in df.iterrows():
    try:
        for each in row['常用药物'].split(','):
            rels_commonddrug.append([row['疾病名称'], each])
    except:
        pass
rels_commonddrug = deduplicate(rels_commonddrug)

In [20]:
#关系：疾病-不可以吃
rels_noteat = []
for idx, row in df.iterrows():
    try:
        for each in row['不可以吃'].split(','):
            rels_noteat.append([row['疾病名称'], each])
    except:
        pass
rels_noteat = deduplicate(rels_noteat)

In [21]:
#关系：疾病-可以吃
rels_doeat = []
for idx, row in df.iterrows():
    try:
        for each in row['可以吃'].split(','):
            rels_doeat.append([row['疾病名称'], each])
    except:
        pass
rels_doeat = deduplicate(rels_doeat)

In [22]:
#关系：疾病-推荐吃
rels_recommandeat = []
for idx, row in df.iterrows():
    try:
        for each in row['推荐吃'].split(','):
            rels_recommandeat.append([row['疾病名称'], each])
    except:
        pass
rels_recommandeat = deduplicate(rels_recommandeat)

In [23]:
#关系：药物厂商-具体药物
rels_drug_producer = []
for each in df['具体药物']:
    try:
        for each_drug in each.split(','):
            producer = each_drug.split('(')[0]
            drug = each_drug.split('(')[1][:-1]
            rels_drug_producer.append([producer, drug])
    except:
        pass
rels_drug_producer = deduplicate(rels_drug_producer)

In [24]:
#关系：疾病-科室、小科室-大科室
rels_category = [] # 关系：疾病-科室
rels_department = [] # 关系：小科室-大科室
for idx, row in df.iterrows():
    if len(row['科室'].split(',')) == 1:
        rels_category.append([row['疾病名称'], row['科室']])
    else:
        big = row['科室'].split(',')[0] # 大科室
        small = row['科室'].split(',')[1] # 小科室
        rels_category.append([row['疾病名称'], small])
        rels_department.append([small, big])
rels_category = deduplicate(rels_category)
rels_department = deduplicate(rels_department)

In [25]:
# 删除所有实体和关系并进行关系建立
g = Graph('neo4j://localhost:7687', auth=('neo4j', 'duan2000814'))
cypher = 'MATCH (n) DETACH DELETE n'
g.run(cypher)

(No data)

In [26]:
#创建疾病实体
count = 0
for disease_dict in disease_infos:
    try:
        node = Node("Disease",
                    name=disease_dict['疾病名称'],
                    desc=disease_dict['疾病描述'],
                    prevent=disease_dict['预防措施'],
                    cause=disease_dict['病因'],
                    easy_get=disease_dict['易感人群'],
                    cure_lasttime=disease_dict['疗程'],
                    cure_department=disease_dict['科室'],
                    cure_way=disease_dict['疗法'], 
                    cured_prob=disease_dict['治愈率'])
        g.create(node)
        count += 1
        print('创建疾病实体：', disease_dict['疾病名称'])
    except:
        pass
print('共创建 {} 个疾病实体'.format(count))

创建疾病实体： 肺泡蛋白质沉积症
创建疾病实体： 百日咳
创建疾病实体： 苯中毒
创建疾病实体： 喘息样支气管炎
创建疾病实体： 成人呼吸窘迫综合征
创建疾病实体： 大量羊水吸入
创建疾病实体： 单纯性肺嗜酸粒细胞浸润症
创建疾病实体： 大叶性肺炎
创建疾病实体： 大楼病综合征
创建疾病实体： 二硫化碳中毒
创建疾病实体： 肺-胸膜阿米巴病
创建疾病实体： 肺出血－肾炎综合征
创建疾病实体： 肺放线菌病
创建疾病实体： 肺泡蛋白沉着症
创建疾病实体： 肺曲菌病
创建疾病实体： 放射性肺炎
创建疾病实体： 肺念珠菌病
创建疾病实体： 肺大疱
创建疾病实体： 肺炎球菌肺炎
创建疾病实体： 肺气肿
创建疾病实体： 肺炎杆菌肺炎
共创建 21 个疾病实体


In [27]:
#创建药物实体
for each in drugs:
    node = Node('Drug', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 注射用盐酸头孢替安
创建实体 蒺藜药酒
创建实体 地塞米松磷酸钠注射液
创建实体 麦迪霉素片
创建实体 雷公藤多苷片
创建实体 复方蛤青胶囊
创建实体 硝酸咪康唑栓
创建实体 环酯红霉素片
创建实体 穿心莲内酯片
创建实体 注射用盐酸氨溴索
创建实体 噻托溴铵粉吸入剂
创建实体 克霉唑阴道片
创建实体 伊曲康唑胶囊
创建实体 琥乙红霉素片
创建实体 乙酰螺旋霉素片
创建实体 奥硝唑片
创建实体 仁青芒觉
创建实体 二羟丙茶碱片
创建实体 诺氟沙星胶囊
创建实体 茶碱缓释胶囊
创建实体 祛痰止咳胶囊
创建实体 喘嗽宁片
创建实体 甲硝唑片
创建实体 盐酸妥洛特罗片
创建实体 氯霉素片
创建实体 甲泼尼龙片
创建实体 云南花粉片
创建实体 头孢拉定胶囊
创建实体 白及颗粒
创建实体 如意定喘丸
创建实体 人血白蛋白
创建实体 红霉素肠溶片
创建实体 注射用还原型谷胱甘肽钠
创建实体 硫酸沙丁胺醇片
创建实体 注射用美洛西林钠舒巴坦钠
创建实体 注射用磺苄西林钠
创建实体 盐酸左氧氟沙星胶囊
创建实体 小青龙合剂
创建实体 盐酸克林霉素棕榈酸酯颗粒
创建实体 盐酸氨溴索葡萄糖注射液
创建实体 注射用头孢唑林钠
创建实体 小青龙颗粒
创建实体 硫酸沙丁胺醇气雾剂
创建实体 注射用鼠神经生长因子
创建实体 硫酸特布他林片
创建实体 噻托溴铵粉雾剂
创建实体 注射用硫代硫酸钠
创建实体 头孢氨苄胶囊
创建实体 头孢氨苄甲氧苄啶片
创建实体 盐酸特比萘芬溶液
创建实体 蛤蚧定喘丸
创建实体 氨苄西林胶囊
创建实体 注射用布美他尼
创建实体 环吡酮胺乳膏
创建实体 大月晶丸
创建实体 盐酸克林霉素胶囊
创建实体 百咳静糖浆
创建实体 阿奇霉素片
创建实体 司帕沙星分散片
创建实体 注射用呋塞米
创建实体 盐酸利多卡因注射液
创建实体 十一味金色丸
创建实体 化痰平喘片
创建实体 蛇胆川贝液
创建实体 托西酸舒他西林胶囊
创建实体 琥乙红霉素颗粒
创建实体 阿莫西林颗粒
创建实体 布美他尼片
创建实体 喷托维林氯化铵糖浆
创建实体 通宣理肺丸
创建实体 肺气肿片
创建实体 盐酸特比萘芬片
创建实体 硝酸益康唑乳膏
创建实体 转移因子口服溶液
创建实体 氨溴特罗口服溶液
创建实体 盐酸克林霉素棕榈酸酯分散片
创建实体 酮康唑乳膏
创建实体 胸腺肽肠溶片
创建实体 联苯苄唑乳膏

In [28]:
#创建食物实体
for each in foods:
    node = Node('Food', name=each)
    g.create(node)

In [39]:
# 删除所有标签为 Food 的节点
delete_query = """
MATCH (f:Food)
DETACH DELETE f
"""

try:
    # 执行删除节点的查询
    g.run(delete_query)
    print("All Food nodes have been deleted.")
except Exception as e:
    print("An error occurred:", e)

All Food nodes have been deleted.


In [29]:
#创建检查实体
for each in checks:
    node = Node('Check', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 白细胞分类计数
创建实体 胸部CT检查
创建实体 痰液病原体检查
创建实体 胸部透视
创建实体 肺活检
创建实体 支气管镜检查
创建实体 小白鼠毒力试验
创建实体 心肺功能运动试验（CPET）
创建实体 通气与血流灌注比值（V/Q）
创建实体 胸部平片
创建实体 肺功能检查
创建实体 深吸气量（IC）
创建实体 痰液细菌培养
创建实体 痰液细菌涂片检查
创建实体 抗链球菌型M蛋白抗体
创建实体 Optochin敏感试验
创建实体 纤维支气管镜检查
创建实体 肺和胸膜听诊
创建实体 痰液常规检查
创建实体 酶联免疫吸附试验
创建实体 神经系统检查
创建实体 耳、鼻、咽拭子细菌培养
创建实体 残气量／肺总量比值（RV/TLC）
创建实体 尿液镜检法
创建实体 痰培养
创建实体 肺容量测定
创建实体 一秒用力呼出量／用力肺活量比值
创建实体 脑电图检查
创建实体 肺活量体重指数
创建实体 免疫电泳
创建实体 最大呼气流量-容积曲线（MEFV）
创建实体 肺活量（VC）
创建实体 肺量计检测
创建实体 心电图
创建实体 支气管造影
创建实体 先令氏指数
创建实体 抗链球菌壁多糖抗体
创建实体 肺毛细血管楔压
创建实体 无效腔气量／潮气量比值
创建实体 痰液中细胞分类
创建实体 钼靶X线检查
创建实体 肌电图
创建实体 痰液中寄生虫和虫卵
创建实体 每分钟最大通气量（MVV）
创建实体 肺部检查
创建实体 隐血试验与含铁血黄素检查
创建实体 肺泡气-动脉血氧分压差
创建实体 眼底荧光血管造影
创建实体 血常规
创建实体 骨髓象分析
创建实体 呼吸肌功能测定
创建实体 尿常规
创建实体 肾功能检查
创建实体 血浆蛋白C抗原
创建实体 周围血白细胞计数及分类检验
创建实体 抗肾小球基底膜抗体测定（AGBM）
创建实体 CT检查


In [30]:
#创建科室实体
for each in departments:
    node = Node('Department', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 急诊科
创建实体 小儿内科
创建实体 内科
创建实体 其他综合
创建实体 儿科
创建实体 呼吸内科
创建实体 其他科室


In [31]:
#创建 药物厂商 实体
for each in producers:
    node = Node('Producer', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 吉尔吉药业喘嗽宁片
创建实体 全星制药注射用盐酸头孢替安
创建实体 北海阳光药业阿奇霉素颗粒
创建实体 仙琚制药克霉唑阴道片
创建实体 西藏甘露仁青芒觉
创建实体 天津太平洋二羟丙茶碱片
创建实体 注射用盐酸头孢替安
创建实体 泰邦生物人血白蛋白
创建实体 云门药业头孢氨苄甲氧苄啶片
创建实体 搏沃欣
创建实体 海力制药阿奇霉素颗粒
创建实体 博利康尼
创建实体 金诃藏药仁青芒觉
创建实体 百利药业阿奇霉素颗粒
创建实体 吉春制药祛痰止咳胶囊
创建实体 辅仁药业阿奇霉素片
创建实体 广东卫伦人血白蛋白
创建实体 吉林东方制药枸橼酸喷托维林
创建实体 晶珠藏药蛇胆川贝液
创建实体 哈药总厂托西酸舒他西林胶囊
创建实体 锦华药业诺氟沙星胶囊
创建实体 傲群
创建实体 帮备
创建实体 鲁抗阿莫西林颗粒
创建实体 朗瑞
创建实体 六安华源制药盐酸氨溴索葡萄
创建实体 鲁抗赛特替硝唑片
创建实体 衡阳紫光古汉硝酸益康唑乳膏
创建实体 苏州三药盐酸克林霉素胶囊
创建实体 康美药业诺氟沙星胶囊
创建实体 央宗药业蒺藜药酒
创建实体 遂成药业地塞米松磷酸钠注射
创建实体 广东华南依托红霉素片
创建实体 奇莫欣
创建实体 贵州汉方药业雷公藤多苷片
创建实体 皇象铁力蓝天甲硝唑维B6片
创建实体 一品红注射用盐酸溴己新
创建实体 山西云鹏制药氨茶碱片
创建实体 诺健
创建实体 阿莫仙阿莫西林颗粒
创建实体 申优
创建实体 海王阿奇霉素片
创建实体 可尔生
创建实体 华瑞联合制药头孢氨苄甲氧苄
创建实体 海力制药替硝唑片
创建实体 南通中华药业硝酸咪康唑乳膏
创建实体 迪沙药业甲硝唑维B6片
创建实体 欣得生
创建实体 锦华药业盐酸克林霉素胶囊
创建实体 湖南九典制药盐酸班布特罗胶
创建实体 安必君氨苄西林胶囊
创建实体 锦华药业氯霉素片
创建实体 东药替硝唑片
创建实体 卓峰地塞米松磷酸钠注射液
创建实体 莎普爱思盐酸氨溴索葡萄糖注
创建实体 白云山天心地塞米松磷酸钠注
创建实体 明水药业头孢氨苄甲氧苄啶片
创建实体 润阳药业盐酸氨溴索葡萄糖注
创建实体 津华晖星替硝唑片
创建实体 彼迪药业阿奇霉素分散片
创建实体 利君沙
创建实体 北京曙光药业红霉素肠溶片
创建实体 赣南制药喷托维林氯化铵糖浆
创建实体 澳利达奈德注射用盐酸溴己新
创建实体 达仁堂通宣理肺丸
创建实体 辽宁

In [32]:
#创建 症状 实体
for each in symptoms:
    node = Node('Symptom', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 咯血伴发热
创建实体 盗汗
创建实体 抽搐
创建实体 发热伴咳嗽、咯...
创建实体 紧张性头晕
创建实体 小支气管粘膜水肿
创建实体 蛋白尿
创建实体 肺部啰音
创建实体 口唇和甲床略带青紫
创建实体 胸闷憋气
创建实体 痰中带血丝
创建实体 哮鸣音
创建实体 横膈低平
创建实体 间歇性头痛
创建实体 惊厥
创建实体 骨膜炎
创建实体 桶状胸
创建实体 纤毛上皮细胞损伤脱落
创建实体 呼吸音减弱
创建实体 咳嗽伴胸痛
创建实体 冬春季的慢性咳...
创建实体 浅感觉减退或缺失
创建实体 感觉障碍
创建实体 鼻塞
创建实体 湿啰音
创建实体 乏力
创建实体 肺阴虚
创建实体 口唇青紫
创建实体 肺纤维化
创建实体 闫鹏辉
创建实体 咳嗽伴哮鸣音
创建实体 心源性呼吸窘迫
创建实体 多发性神经炎
创建实体 呼气音延长
创建实体 畏寒
创建实体 咯血
创建实体 肺纹理增粗
创建实体 胸痛
创建实体 间歇性头晕
创建实体 呼吸困难
创建实体 肺泡灌洗液可见...
创建实体 咽部异物感
创建实体 肺泡炎症
创建实体 喘息
创建实体 腱反射消失
创建实体 体重减轻
创建实体 咳出棕色痰栓
创建实体 低热
创建实体 黏稠或脓性痰伴...
创建实体 耸肩喘息
创建实体 紧张性头痛
创建实体 发烧
创建实体 紫绀
创建实体 昏迷
创建实体 痰呈粘液脓性
创建实体 化学性支气管炎
创建实体 腹泻
创建实体 纵隔浊音界扩大
创建实体 胸闷
创建实体 呕吐
创建实体 急性面容
创建实体 出生后即有持续青紫
创建实体 谵妄
创建实体 眼睛痒
创建实体 闫铁
创建实体 嗜酸性粒细胞增多
创建实体 目赤
创建实体 面色青紫
创建实体 头晕
创建实体 恶心
创建实体 变应性咳嗽
创建实体 痉挛性咳嗽
创建实体 毓卓
创建实体 气急
创建实体 肺部出血
创建实体 气短
创建实体 咳铁锈色痰
创建实体 吸气时有蝉鸣音


In [33]:
#创建知识图谱关系（连接、边）
def create_relationship(start_node, end_node, edges, rel_type, rel_name):
    '''创建关系函数'''
    for edge in edges:
        p = edge[0]
        q = edge[1]
        # 创建关系的 Cypher 语句
        query = "match(p:%s),(q:%s) where p.name='%s' and q.name='%s' create (p)-[rel:%s{name:'%s'}]->(q)" % (start_node, end_node, p, q, rel_type, rel_name)
        try:
            g.run(query) # 运行 Cypher 语句
            print('创建关系 {}-{}->{}'.format(p, rel_type, q))
        except Exception as e:
            print(e)

In [34]:
create_relationship('Disease', 'Food', rels_recommandeat, 'recommand_eat', '推荐食谱')
create_relationship('Disease', 'Food', rels_noteat, 'no_eat', '忌吃')
create_relationship('Disease', 'Food', rels_doeat, 'do_eat', '宜吃')
create_relationship('Department', 'Department', rels_department, 'belongs_to', '属于')
create_relationship('Disease', 'Drug', rels_commonddrug, 'common_drug', '常用药品')
create_relationship('Producer', 'Drug', rels_drug_producer, 'drugs_of', '生产药品')
create_relationship('Disease', 'Drug', rels_recommanddrug, 'recommand_drug', '好评药品')
create_relationship('Disease', 'Check', rels_check, 'need_check', '诊断检查')
create_relationship('Disease', 'Symptom', rels_symptom, 'has_symptom', '症状')
create_relationship('Disease', 'Disease', rels_acompany, 'acompany_with', '并发症')
create_relationship('Disease', 'Department', rels_category, 'belongs_to', '所属科室')

创建关系 百日咳-recommand_eat->清蒸鸡蛋羹
创建关系 百日咳-recommand_eat->百合双耳鸡蛋羹
创建关系 百日咳-recommand_eat->排骨汤
创建关系 百日咳-recommand_eat->罗汉果雪耳鸡汤
创建关系 百日咳-recommand_eat->小黄瓜凉拌面
创建关系 百日咳-recommand_eat->黄瓜三丝汤
创建关系 百日咳-recommand_eat->黄瓜拌兔丝
创建关系 百日咳-recommand_eat->黄瓜拌皮丝
创建关系 苯中毒-recommand_eat->豆腐干炒韭菜
创建关系 苯中毒-recommand_eat->素炒小白菜
创建关系 苯中毒-recommand_eat->白菜蛋花粥
创建关系 喘息样支气管炎-recommand_eat->紫苏粥
创建关系 喘息样支气管炎-recommand_eat->菊花芦根茶
创建关系 喘息样支气管炎-recommand_eat->生芦根粥
创建关系 喘息样支气管炎-recommand_eat->鲜芦根粥
创建关系 喘息样支气管炎-recommand_eat->薏仁党参粥
创建关系 喘息样支气管炎-recommand_eat->党参粥
创建关系 喘息样支气管炎-recommand_eat->黄芪粥
创建关系 喘息样支气管炎-recommand_eat->鸡蛋木耳粥
创建关系 成人呼吸窘迫综合征-recommand_eat->百合糖粥
创建关系 成人呼吸窘迫综合征-recommand_eat->百合粥
创建关系 成人呼吸窘迫综合征-recommand_eat->百合花生汤
创建关系 成人呼吸窘迫综合征-recommand_eat->首乌百合粥
创建关系 成人呼吸窘迫综合征-recommand_eat->薏米莲子粥
创建关系 成人呼吸窘迫综合征-recommand_eat->牛奶玉米汤
创建关系 成人呼吸窘迫综合征-recommand_eat->萝卜豆腐汤
创建关系 成人呼吸窘迫综合征-recommand_eat->瓜粒杂锦汤
创建关系 大叶性肺炎-recommand_eat->奶汤锅子鱼
创建关系 大叶性肺炎-recommand_eat->酱豆腐汁烧猪肉
创建关系 大叶性肺炎-recommand_eat->百合汤
创建关系 大叶性肺炎-recommand_

创建关系 利君沙-drugs_of->琥乙红霉素颗粒
创建关系 桂林南药布美他尼片-drugs_of->布美他尼片
创建关系 雄巴拉曲神水十一味金色丸-drugs_of->十一味金色丸
创建关系 西藏甘露仁青芒觉-drugs_of->仁青芒觉
创建关系 金诃藏药大月晶丸-drugs_of->大月晶丸
创建关系 金诃藏药仁青芒觉-drugs_of->仁青芒觉
创建关系 苏肽生-drugs_of->注射用鼠神经生长因子
创建关系 新峰药业盐酸利多卡因注射液-drugs_of->盐酸利多卡因注射液
创建关系 海斯制药盐酸利多卡因注射液-drugs_of->盐酸利多卡因注射液
创建关系 西藏甘露十一味金色丸-drugs_of->十一味金色丸
创建关系 卓峰地塞米松磷酸钠注射液-drugs_of->地塞米松磷酸钠注射液
创建关系 白云山天心地塞米松磷酸钠注-drugs_of->地塞米松磷酸钠注射液
创建关系 涟水制药地塞米松磷酸钠注射-drugs_of->地塞米松磷酸钠注射液
创建关系 金耀药业地塞米松磷酸钠注射-drugs_of->地塞米松磷酸钠注射液
创建关系 遂成药业地塞米松磷酸钠注射-drugs_of->地塞米松磷酸钠注射液
创建关系 恩经复-drugs_of->注射用鼠神经生长因子
创建关系 金路捷-drugs_of->注射用鼠神经生长因子
创建关系 新亚注射用硫代硫酸钠-drugs_of->注射用硫代硫酸钠
创建关系 北京永康药业盐酸利多卡因注-drugs_of->盐酸利多卡因注射液
创建关系 北京益民药业盐酸利多卡因注-drugs_of->盐酸利多卡因注射液
创建关系 中国大冢盐酸利多卡因注射液-drugs_of->盐酸利多卡因注射液
创建关系 盐酸利多卡因注射液-drugs_of->盐酸利多卡因注射液
创建关系 全星制药注射用布美他尼-drugs_of->注射用布美他尼
创建关系 皇隆制药注射用呋塞米-drugs_of->注射用呋塞米
创建关系 信谊布地奈德气雾剂-drugs_of->布地奈德气雾剂
创建关系 桑海制药喷托维林氯化铵糖浆-drugs_of->喷托维林氯化铵糖浆
创建关系 赣南制药喷托维林氯化铵糖浆-drugs_of->喷托维林氯化铵糖浆
创建关系 奇莫欣-drugs_of->胸腺肽肠溶片
创建关系 振兴氨茶碱片-drugs_of->氨茶碱片
创

创建关系 江西诚志永丰蛇胆川贝液-drugs_of->蛇胆川贝液
创建关系 鲁抗阿莫西林颗粒-drugs_of->阿莫西林颗粒
创建关系 广东华南依托红霉素片-drugs_of->依托红霉素片
创建关系 孩尔欣头孢克洛颗粒-drugs_of->头孢克洛颗粒
创建关系 银力舒-drugs_of->头孢丙烯分散片
创建关系 达仁堂通宣理肺丸-drugs_of->通宣理肺丸
创建关系 申优-drugs_of->头孢拉定分散片
创建关系 康良头孢拉定胶囊-drugs_of->头孢拉定胶囊
创建关系 安必君氨苄西林胶囊-drugs_of->氨苄西林胶囊
创建关系 佳乐弗乳酸左氧氟沙星片-drugs_of->乳酸左氧氟沙星片
创建关系 银诺克肺宝蛤蚧定喘丸-drugs_of->蛤蚧定喘丸
创建关系 达力芬-drugs_of->头孢克肟颗粒
创建关系 鲁神乙酰螺旋霉素片-drugs_of->乙酰螺旋霉素片
创建关系 鲁抗注射用头孢唑林钠-drugs_of->注射用头孢唑林钠
创建关系 上海新亚头孢克肟分散片-drugs_of->头孢克肟分散片
创建关系 鲁抗麦迪霉素片-drugs_of->麦迪霉素片
创建关系 珠海联邦头孢拉定胶囊-drugs_of->头孢拉定胶囊
创建关系 朗瑞-drugs_of->司帕沙星分散片
创建关系 百日咳-recommand_drug->琥乙红霉素片
创建关系 百日咳-recommand_drug->琥乙红霉素颗粒
创建关系 百日咳-recommand_drug->百咳静糖浆
创建关系 百日咳-recommand_drug->穿心莲内酯片
创建关系 百日咳-recommand_drug->红霉素肠溶片
创建关系 百日咳-recommand_drug->环酯红霉素片
创建关系 苯中毒-recommand_drug->布美他尼片
创建关系 苯中毒-recommand_drug->十一味金色丸
创建关系 苯中毒-recommand_drug->注射用布美他尼
创建关系 苯中毒-recommand_drug->注射用呋塞米
创建关系 苯中毒-recommand_drug->盐酸利多卡因注射液
创建关系 苯中毒-recommand_drug->注射用硫代硫酸钠
创建关系 苯中毒-recommand_drug->大月晶丸
创建关系 苯中毒-recommand_dr

创建关系 百日咳-has_symptom->胸闷
创建关系 百日咳-has_symptom->肺阴虚
创建关系 百日咳-has_symptom->抽搐
创建关系 百日咳-has_symptom->低热
创建关系 百日咳-has_symptom->闫鹏辉
创建关系 百日咳-has_symptom->惊厥
创建关系 苯中毒-has_symptom->恶心
创建关系 苯中毒-has_symptom->抽搐
创建关系 苯中毒-has_symptom->感觉障碍
创建关系 喘息样支气管炎-has_symptom->耸肩喘息
创建关系 喘息样支气管炎-has_symptom->哮鸣音
创建关系 喘息样支气管炎-has_symptom->纤毛上皮细胞损伤脱落
创建关系 喘息样支气管炎-has_symptom->变应性咳嗽
创建关系 喘息样支气管炎-has_symptom->化学性支气管炎
创建关系 喘息样支气管炎-has_symptom->喘息
创建关系 喘息样支气管炎-has_symptom->冬春季的慢性咳...
创建关系 喘息样支气管炎-has_symptom->咳嗽伴哮鸣音
创建关系 成人呼吸窘迫综合征-has_symptom->呼吸困难
创建关系 成人呼吸窘迫综合征-has_symptom->紫绀
创建关系 成人呼吸窘迫综合征-has_symptom->心源性呼吸窘迫
创建关系 大量羊水吸入-has_symptom->面色青紫
创建关系 大量羊水吸入-has_symptom->呼吸困难
创建关系 大量羊水吸入-has_symptom->口唇青紫
创建关系 大量羊水吸入-has_symptom->肺纹理增粗
创建关系 大量羊水吸入-has_symptom->出生后即有持续青紫
创建关系 大量羊水吸入-has_symptom->口唇和甲床略带青紫
创建关系 大量羊水吸入-has_symptom->气急
创建关系 单纯性肺嗜酸粒细胞浸润症-has_symptom->嗜酸性粒细胞增多
创建关系 单纯性肺嗜酸粒细胞浸润症-has_symptom->咽部异物感
创建关系 单纯性肺嗜酸粒细胞浸润症-has_symptom->胸闷憋气
创建关系 单纯性肺嗜酸粒细胞浸润症-has_symptom->咯血伴发热
创建关系 单纯性肺嗜酸粒细胞浸润症-has_symptom->胸闷
创建关系 

# 构建问答机器人

## 1. 构建类AnswerSearche：执行查询，生成回答

In [51]:
#搜索答案
class AnswerSearcher:
    def __init__(self):
        # 替换知识图谱信息
        self.g = Graph('bolt://localhost:7687', auth=('neo4j', 'duan2000814'))
        self.num_limit = 20

    '''执行cypher查询，并返回相应结果'''
    def search_main(self, sqls):
        final_answers = []
        for sql_ in sqls:
            question_type = sql_['question_type']
            queries = sql_['sql']
            answers = []
            for query in queries:
                ress = self.g.run(query).data()
                answers += ress
            final_answer = self.answer_prettify(question_type, answers)
            if final_answer:
                final_answers.append(final_answer)
        return final_answers

    '''根据对应的qustion_type，调用相应的回复模板'''
    def answer_prettify(self, question_type, answers):
        final_answer = []
        if not answers:
            return ''
        if question_type == 'disease_symptom':
            desc = [i['n.name'] for i in answers]
            subject = answers[0]['m.name']
            final_answer = '{0}的症状包括：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

        elif question_type == 'symptom_disease':
            desc = [i['m.name'] for i in answers]
            subject = answers[0]['n.name']
            final_answer = '症状{0}可能染上的疾病有：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

        elif question_type == 'disease_cause':
            desc = [i['m.cause'] for i in answers]
            subject = answers[0]['m.name']
            final_answer = '{0}可能的成因有：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

        elif question_type == 'disease_prevent':
            desc = [i['m.prevent'] for i in answers]
            subject = answers[0]['m.name']
            final_answer = '{0}的预防措施包括：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

        elif question_type == 'disease_lasttime':
            desc = [i['m.cure_lasttime'] for i in answers]
            subject = answers[0]['m.name']
            final_answer = '{0}治疗可能持续的周期为：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

        elif question_type == 'disease_cureway':
            desc = [';'.join(i['m.cure_way']) for i in answers]
            subject = answers[0]['m.name']
            final_answer = '{0}可以尝试如下治疗：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

        elif question_type == 'disease_cureprob':
            desc = [i['m.cured_prob'] for i in answers]
            subject = answers[0]['m.name']
            final_answer = '{0}治愈的概率为（仅供参考）：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

        elif question_type == 'disease_easyget':
            desc = [i['m.easy_get'] for i in answers]
            subject = answers[0]['m.name']

            final_answer = '{0}的易感人群包括：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

        elif question_type == 'disease_desc':
            desc = [i['m.desc'] for i in answers]
            subject = answers[0]['m.name']
            final_answer = '{0},熟悉一下：{1}'.format(subject,  '；'.join(list(set(desc))[:self.num_limit]))

        elif question_type == 'disease_acompany':
            desc1 = [i['n.name'] for i in answers]
            desc2 = [i['m.name'] for i in answers]
            subject = answers[0]['m.name']
            desc = [i for i in desc1 + desc2 if i != subject]
            final_answer = '{0}的症状包括：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

        elif question_type == 'disease_not_food':
            desc = [i['n.name'] for i in answers]
            subject = answers[0]['m.name']
            final_answer = '{0}忌食的食物包括有：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

        elif question_type == 'disease_do_food':
            do_desc = [i['n.name'] for i in answers if i['r.name'] == '宜吃']
            recommand_desc = [i['n.name'] for i in answers if i['r.name'] == '推荐食谱']
            subject = answers[0]['m.name']
            final_answer = '{0}宜食的食物包括有：{1}\n推荐食谱包括有：{2}'.format(subject, ';'.join(list(set(do_desc))[:self.num_limit]), ';'.join(list(set(recommand_desc))[:self.num_limit]))

        elif question_type == 'food_not_disease':
            desc = [i['m.name'] for i in answers]
            subject = answers[0]['n.name']
            final_answer = '患有{0}的人最好不要吃{1}'.format('；'.join(list(set(desc))[:self.num_limit]), subject)

        elif question_type == 'food_do_disease':
            desc = [i['m.name'] for i in answers]
            subject = answers[0]['n.name']
            final_answer = '患有{0}的人建议多试试{1}'.format('；'.join(list(set(desc))[:self.num_limit]), subject)

        elif question_type == 'disease_drug':
            desc = [i['n.name'] for i in answers]
            subject = answers[0]['m.name']
            final_answer = '{0}通常的使用的药品包括：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

        elif question_type == 'drug_disease':
            desc = [i['m.name'] for i in answers]
            subject = answers[0]['n.name']
            final_answer = '{0}主治的疾病有{1},可以试试'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

        elif question_type == 'disease_check':
            desc = [i['n.name'] for i in answers]
            subject = answers[0]['m.name']
            final_answer = '{0}通常可以通过以下方式检查出来：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

        elif question_type == 'check_disease':
            desc = [i['m.name'] for i in answers]
            subject = answers[0]['n.name']
            final_answer = '通常可以通过{0}检查出来的疾病有{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

        return final_answer

searcher = AnswerSearcher()

## 构建类QuestionPaser：问题解析，生成查询语句

In [36]:
#question_parser.py
class QuestionPaser:

    '''构建实体节点'''
    def build_entitydict(self, args):
        entity_dict = {}
        for arg, types in args.items():
            for type in types:
                if type not in entity_dict:
                    entity_dict[type] = [arg]
                else:
                    entity_dict[type].append(arg)

        return entity_dict

    '''解析主函数'''
    def parser_main(self, res_classify):
        args = res_classify['args']
        entity_dict = self.build_entitydict(args)
        question_types = res_classify['question_types']
        sqls = []
        for question_type in question_types:
            sql_ = {}
            sql_['question_type'] = question_type
            sql = []
            if question_type == 'disease_symptom':
                sql = self.sql_transfer(question_type, entity_dict.get('disease'))

            elif question_type == 'symptom_disease':
                sql = self.sql_transfer(question_type, entity_dict.get('symptom'))

            elif question_type == 'disease_cause':
                sql = self.sql_transfer(question_type, entity_dict.get('disease'))

            elif question_type == 'disease_acompany':
                sql = self.sql_transfer(question_type, entity_dict.get('disease'))

            elif question_type == 'disease_not_food':
                sql = self.sql_transfer(question_type, entity_dict.get('disease'))

            elif question_type == 'disease_do_food':
                sql = self.sql_transfer(question_type, entity_dict.get('disease'))

            elif question_type == 'food_not_disease':
                sql = self.sql_transfer(question_type, entity_dict.get('food'))

            elif question_type == 'food_do_disease':
                sql = self.sql_transfer(question_type, entity_dict.get('food'))

            elif question_type == 'disease_drug':
                sql = self.sql_transfer(question_type, entity_dict.get('disease'))

            elif question_type == 'drug_disease':
                sql = self.sql_transfer(question_type, entity_dict.get('drug'))

            elif question_type == 'disease_check':
                sql = self.sql_transfer(question_type, entity_dict.get('disease'))

            elif question_type == 'check_disease':
                sql = self.sql_transfer(question_type, entity_dict.get('check'))

            elif question_type == 'disease_prevent':
                sql = self.sql_transfer(question_type, entity_dict.get('disease'))

            elif question_type == 'disease_lasttime':
                sql = self.sql_transfer(question_type, entity_dict.get('disease'))

            elif question_type == 'disease_cureway':
                sql = self.sql_transfer(question_type, entity_dict.get('disease'))

            elif question_type == 'disease_cureprob':
                sql = self.sql_transfer(question_type, entity_dict.get('disease'))

            elif question_type == 'disease_easyget':
                sql = self.sql_transfer(question_type, entity_dict.get('disease'))

            elif question_type == 'disease_desc':
                sql = self.sql_transfer(question_type, entity_dict.get('disease'))

            if sql:
                sql_['sql'] = sql

                sqls.append(sql_)

        return sqls

    '''针对不同的问题，分开进行处理'''
    def sql_transfer(self, question_type, entities):
        if not entities:
            return []

        # 查询语句
        sql = []
        # 查询疾病的原因
        if question_type == 'disease_cause':
            sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.cause".format(i) for i in entities]

        # 查询疾病的防御措施
        elif question_type == 'disease_prevent':
            sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.prevent".format(i) for i in entities]

        # 查询疾病的持续时间
        elif question_type == 'disease_lasttime':
            sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.cure_lasttime".format(i) for i in entities]

        # 查询疾病的治愈概率
        elif question_type == 'disease_cureprob':
            sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.cured_prob".format(i) for i in entities]

        # 查询疾病的治疗方式
        elif question_type == 'disease_cureway':
            sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.cure_way".format(i) for i in entities]

        # 查询疾病的易发人群
        elif question_type == 'disease_easyget':
            sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.easy_get".format(i) for i in entities]

        # 查询疾病的相关介绍
        elif question_type == 'disease_desc':
            sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.desc".format(i) for i in entities]

        # 查询疾病有哪些症状
        elif question_type == 'disease_symptom':
            sql = ["MATCH (m:Disease)-[r:has_symptom]->(n:Symptom) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

        # 查询症状会导致哪些疾病
        elif question_type == 'symptom_disease':
            sql = ["MATCH (m:Disease)-[r:has_symptom]->(n:Symptom) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

        # 查询疾病的并发症
        elif question_type == 'disease_acompany':
            sql1 = ["MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
            sql2 = ["MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
            sql = sql1 + sql2
        # 查询疾病的忌口
        elif question_type == 'disease_not_food':
            sql = ["MATCH (m:Disease)-[r:no_eat]->(n:Food) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

        # 查询疾病建议吃的东西
        elif question_type == 'disease_do_food':
            sql1 = ["MATCH (m:Disease)-[r:do_eat]->(n:Food) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
            sql2 = ["MATCH (m:Disease)-[r:recommand_eat]->(n:Food) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
            sql = sql1 + sql2

        # 已知忌口查疾病
        elif question_type == 'food_not_disease':
            sql = ["MATCH (m:Disease)-[r:no_eat]->(n:Food) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

        # 已知推荐查疾病
        elif question_type == 'food_do_disease':
            sql1 = ["MATCH (m:Disease)-[r:do_eat]->(n:Food) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
            sql2 = ["MATCH (m:Disease)-[r:recommand_eat]->(n:Food) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
            sql = sql1 + sql2

        # 查询疾病常用药品－药品别名记得扩充
        elif question_type == 'disease_drug':
            sql1 = ["MATCH (m:Disease)-[r:common_drug]->(n:Drug) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
            sql2 = ["MATCH (m:Disease)-[r:recommand_drug]->(n:Drug) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
            sql = sql1 + sql2

        # 已知药品查询能够治疗的疾病
        elif question_type == 'drug_disease':
            sql1 = ["MATCH (m:Disease)-[r:common_drug]->(n:Drug) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
            sql2 = ["MATCH (m:Disease)-[r:recommand_drug]->(n:Drug) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
            sql = sql1 + sql2
        # 查询疾病应该进行的检查
        elif question_type == 'disease_check':
            sql = ["MATCH (m:Disease)-[r:need_check]->(n:Check) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

        # 已知检查查询疾病
        elif question_type == 'check_disease':
            sql = ["MATCH (m:Disease)-[r:need_check]->(n:Check) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

        return sql

# 构建QuestionClassifier：进行问题分配

In [37]:
#question_classifier.py
import os
import ahocorasick

class QuestionClassifier:
    def __init__(self):
        # cur_dir = '/'.join(os.path.abspath(__file__).split('/')[:-1])
        cur_dir = os.getcwd()
        #　特征词路径
        self.disease_path = os.path.join(cur_dir, 'dict/disease.txt')
        self.department_path = os.path.join(cur_dir, 'dict/department.txt')
        self.check_path = os.path.join(cur_dir, 'dict/check.txt')
        self.drug_path = os.path.join(cur_dir, 'dict/drug.txt')
        self.food_path = os.path.join(cur_dir, 'dict/food.txt')
        self.producer_path = os.path.join(cur_dir, 'dict/producer.txt')
        self.symptom_path = os.path.join(cur_dir, 'dict/symptom.txt')
        self.deny_path = os.path.join(cur_dir, 'dict/deny.txt')
        # 加载特征词
        self.disease_wds= [i.strip() for i in open(self.disease_path,encoding="utf-8") if i.strip()]#encoding="utf-8"
        self.department_wds= [i.strip() for i in open(self.department_path,encoding="utf-8") if i.strip()]
        self.check_wds= [i.strip() for i in open(self.check_path,encoding="utf-8") if i.strip()]
        self.drug_wds= [i.strip() for i in open(self.drug_path,encoding="utf-8") if i.strip()]
        self.food_wds= [i.strip() for i in open(self.food_path,encoding="utf-8") if i.strip()]
        self.producer_wds= [i.strip() for i in open(self.producer_path,encoding="utf-8") if i.strip()]
        self.symptom_wds= [i.strip() for i in open(self.symptom_path,encoding="utf-8") if i.strip()]
        self.region_words = set(self.department_wds + self.disease_wds + self.check_wds + self.drug_wds + self.food_wds + self.producer_wds + self.symptom_wds)
        self.deny_words = [i.strip() for i in open(self.deny_path,encoding="utf-8") if i.strip()]
        # 构造领域actree
        self.region_tree = self.build_actree(list(self.region_words))
        # 构建词典
        self.wdtype_dict = self.build_wdtype_dict()
        # 问句疑问词
        self.symptom_qwds = ['症状', '表征', '现象', '症候', '表现']
        self.cause_qwds = ['原因','成因', '为什么', '怎么会', '怎样才', '咋样才', '怎样会', '如何会', '为啥', '为何', '如何才会', '怎么才会', '会导致', '会造成']
        self.acompany_qwds = ['并发症', '并发', '一起发生', '一并发生', '一起出现', '一并出现', '一同发生', '一同出现', '伴随发生', '伴随', '共现']
        self.food_qwds = ['饮食', '饮用', '吃', '食', '伙食', '膳食', '喝', '菜' ,'忌口', '补品', '保健品', '食谱', '菜谱', '食用', '食物','补品']
        self.drug_qwds = ['药', '药品', '用药', '胶囊', '口服液', '炎片']
        self.prevent_qwds = ['预防', '防范', '抵制', '抵御', '防止','躲避','逃避','避开','免得','逃开','避开','避掉','躲开','躲掉','绕开',
                             '怎样才能不', '怎么才能不', '咋样才能不','咋才能不', '如何才能不',
                             '怎样才不', '怎么才不', '咋样才不','咋才不', '如何才不',
                             '怎样才可以不', '怎么才可以不', '咋样才可以不', '咋才可以不', '如何可以不',
                             '怎样才可不', '怎么才可不', '咋样才可不', '咋才可不', '如何可不']
        self.lasttime_qwds = ['周期', '多久', '多长时间', '多少时间', '几天', '几年', '多少天', '多少小时', '几个小时', '多少年']
        self.cureway_qwds = ['怎么治疗', '如何医治', '怎么医治', '怎么治', '怎么医', '如何治', '医治方式', '疗法', '咋治', '怎么办', '咋办', '咋治']
        self.cureprob_qwds = ['多大概率能治好', '多大几率能治好', '治好希望大么', '几率', '几成', '比例', '可能性', '能治', '可治', '可以治', '可以医']
        self.easyget_qwds = ['易感人群', '容易感染', '易发人群', '什么人', '哪些人', '感染', '染上', '得上']
        self.check_qwds = ['检查', '检查项目', '查出', '检查', '测出', '试出']
        self.belong_qwds = ['属于什么科', '属于', '什么科', '科室']
        self.cure_qwds = ['治疗什么', '治啥', '治疗啥', '医治啥', '治愈啥', '主治啥', '主治什么', '有什么用', '有何用', '用处', '用途',
                          '有什么好处', '有什么益处', '有何益处', '用来', '用来做啥', '用来作甚', '需要', '要']

        print('model init finished ......')

        return

    '''分类主函数'''
    def classify(self, question):
        data = {}
        medical_dict = self.check_medical(question)
        if not medical_dict:
            return {}
        data['args'] = medical_dict
        #收集问句当中所涉及到的实体类型
        types = []
        for type_ in medical_dict.values():
            types += type_
        question_type = 'others'

        question_types = []

        # 症状
        if self.check_words(self.symptom_qwds, question) and ('disease' in types):
            question_type = 'disease_symptom'
            question_types.append(question_type)

        if self.check_words(self.symptom_qwds, question) and ('symptom' in types):
            question_type = 'symptom_disease'
            question_types.append(question_type)

        # 原因
        if self.check_words(self.cause_qwds, question) and ('disease' in types):
            question_type = 'disease_cause'
            question_types.append(question_type)
        # 并发症
        if self.check_words(self.acompany_qwds, question) and ('disease' in types):
            question_type = 'disease_acompany'
            question_types.append(question_type)

        # 推荐食品
        if self.check_words(self.food_qwds, question) and 'disease' in types:
            deny_status = self.check_words(self.deny_words, question)
            if deny_status:
                question_type = 'disease_not_food'
            else:
                question_type = 'disease_do_food'
            question_types.append(question_type)

        #已知食物找疾病
        if self.check_words(self.food_qwds+self.cure_qwds, question) and 'food' in types:
            deny_status = self.check_words(self.deny_words, question)
            if deny_status:
                question_type = 'food_not_disease'
            else:
                question_type = 'food_do_disease'
            question_types.append(question_type)

        # 推荐药品
        if self.check_words(self.drug_qwds, question) and 'disease' in types:
            question_type = 'disease_drug'
            question_types.append(question_type)

        # 药品治啥病
        if self.check_words(self.cure_qwds, question) and 'drug' in types:
            question_type = 'drug_disease'
            question_types.append(question_type)

        # 疾病接受检查项目
        if self.check_words(self.check_qwds, question) and 'disease' in types:
            question_type = 'disease_check'
            question_types.append(question_type)

        # 已知检查项目查相应疾病
        if self.check_words(self.check_qwds+self.cure_qwds, question) and 'check' in types:
            question_type = 'check_disease'
            question_types.append(question_type)

        #　症状防御
        if self.check_words(self.prevent_qwds, question) and 'disease' in types:
            question_type = 'disease_prevent'
            question_types.append(question_type)

        # 疾病医疗周期
        if self.check_words(self.lasttime_qwds, question) and 'disease' in types:
            question_type = 'disease_lasttime'
            question_types.append(question_type)

        # 疾病治疗方式
        if self.check_words(self.cureway_qwds, question) and 'disease' in types:
            question_type = 'disease_cureway'
            question_types.append(question_type)

        # 疾病治愈可能性
        if self.check_words(self.cureprob_qwds, question) and 'disease' in types:
            question_type = 'disease_cureprob'
            question_types.append(question_type)

        # 疾病易感染人群
        if self.check_words(self.easyget_qwds, question) and 'disease' in types :
            question_type = 'disease_easyget'
            question_types.append(question_type)

        # 若没有查到相关的外部查询信息，那么则将该疾病的描述信息返回
        if question_types == [] and 'disease' in types:
            question_types = ['disease_desc']

        # 若没有查到相关的外部查询信息，那么则将该疾病的描述信息返回
        if question_types == [] and 'symptom' in types:
            question_types = ['symptom_disease']

        # 将多个分类结果进行合并处理，组装成一个字典
        data['question_types'] = question_types

        return data

    '''构造词对应的类型'''
    def build_wdtype_dict(self):
        wd_dict = dict()
        for wd in self.region_words:
            wd_dict[wd] = []
            if wd in self.disease_wds:
                wd_dict[wd].append('disease')
            if wd in self.department_wds:
                wd_dict[wd].append('department')
            if wd in self.check_wds:
                wd_dict[wd].append('check')
            if wd in self.drug_wds:
                wd_dict[wd].append('drug')
            if wd in self.food_wds:
                wd_dict[wd].append('food')
            if wd in self.symptom_wds:
                wd_dict[wd].append('symptom')
            if wd in self.producer_wds:
                wd_dict[wd].append('producer')
        return wd_dict

    '''构造actree，加速过滤'''
    def build_actree(self, wordlist):
        actree = ahocorasick.Automaton()
        for index, word in enumerate(wordlist):
            actree.add_word(word, (index, word))
        actree.make_automaton()
        return actree

    '''问句过滤'''
    def check_medical(self, question):
        region_wds = []
        for i in self.region_tree.iter(question):
            wd = i[1][1]
            region_wds.append(wd)
        stop_wds = []
        for wd1 in region_wds:
            for wd2 in region_wds:
                if wd1 in wd2 and wd1 != wd2:
                    stop_wds.append(wd1)
        final_wds = [i for i in region_wds if i not in stop_wds]
        final_dict = {i:self.wdtype_dict.get(i) for i in final_wds}

        return final_dict

    '''基于特征词进行分类'''
    def check_words(self, wds, sent):
        for wd in wds:
            if wd in sent:
                return True
        return False

In [40]:
os.chdir('QAMedicalKG')


In [41]:
#实例化函数
classifier = QuestionClassifier()

parser = QuestionPaser()

searcher = AnswerSearcher()

model init finished ......


In [53]:
#输入问题
question = '胸闷不能吃什么'

In [54]:
#从问题中提取实体和关系
res_classify = classifier.classify(question)
res_classify


{'args': {'胸闷': ['symptom']}, 'question_types': ['symptom_disease']}

In [55]:
#问题解析-生成Cypher查询语句
res_cypher = parser.parser_main(res_classify)
res_cypher

[{'question_type': 'symptom_disease',
  'sql': ["MATCH (m:Disease)-[r:has_symptom]->(n:Symptom) where n.name = '胸闷' return m.name, r.name, n.name"]}]

In [56]:
#生成回答：
final_answers = searcher.search_main(res_cypher)
final_answers

['症状胸闷可能染上的疾病有：肺气肿；百日咳；肺大疱；单纯性肺嗜酸粒细胞浸润症']